DATA ANALYST / SCIENTIST ENTRY TASK

Taking the following dataset:
Scotch Whiskies Data
Paper: Lapointe, F.-J. & P. Legendre. 1994. A classification of pure malt Scotch whiskies. Applied Statistics 43: 237-257
http://adn.biol.umontreal.ca/~numericalecology/data/scotch.html
(direct link: adn.biol.umontreal.ca/~numericalecology/labo/Scotch/ScotchData.zip)

Please provide meaningful insights, e.g.:
-	What interesting information user can find in the dataset?
-	Choose whisky name and find other whisky(-ies) which you might like as well; Please provide reasoning of your steps
-	How chosen whisky(-ies) can be perceived vs. other types?
-	What factors impact results the most?

Requirements:
-	solution must contain comments (e.g. choice of model(s), assumptions, corrections, conclusions etc.)
-	solution written in python (especially Jupyter Notebook) 
-	solution with commits history on the Github
-	solution (if accepted) has to be presented during the following interview session - Q&A part after presentation


In [2]:
import numpy as np
import pandas as pd

In [3]:
data = pd.read_csv('data/scotch.csv', error_bad_lines=False, delimiter=';')

In [5]:
data.head(5)

,NAME,color wyne,color yellow,color v.pale,color pale,color p.gold,color gold,color o.gold,color f.gold,color bronze,...,DISTRICT,islay,midland,spey,east,west,north,lowland,campbell,islands
0,Aberfeldy,0,1,0,0,0,0,0,0,0,...,MIDLAND,0,1,0,0,0,0,0,0,0
1,Aberlour,0,0,0,0,0,0,0,0,0,...,SPEY,0,0,1,0,0,0,0,0,0
2,Ardberg,0,0,0,0,0,0,0,0,0,...,SOUTH,1,0,0,0,0,0,0,0,0
3,Ardmore,0,0,0,0,1,0,0,0,0,...,SPEY,0,0,1,0,0,0,0,0,0
4,Auchentoshan,0,0,0,0,1,0,0,0,0,...,WEST,0,0,0,0,0,0,1,0,0
